<a href="https://colab.research.google.com/github/phi1z/1yanagiLab/blob/main/N_log_Norm_fitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 対数正規分布の多重フィッティングを行うWebUIです
## 利用方法

1.   ディレクトリー名(`output_dir`)を入力して実行。左のフォルダーにdataなどが作成されます
2.   data/[ディレクトリー名]/にデータ(txtファイル)をアップロード。実行して**ファイル一覧**を確認します
3.   解析するファイルを番号で指定。番号は上に表示されている**ファイル一覧**を参照してください
4.   分布の個数(`weight`)と初期値(`Initial_guess`)を設定してフィッティングを実行。
5.   結果を保存。graphにはグラフが、Fitting_aramsにはパラメーターが保存されています。

※ フィッティングが上手くいかない場合は、初期値を変更してください

## 機能

*   解析するファイルの設定で、すべてのファイルを解析することも可能です。`is_all`にチェックを入れて下さい。
*   結果を削除で、出力結果を削除することができます。また、データの削除も可能です。


In [ ]:
#@title ディレクトリー設定
# ====ここから変更しない====
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import lognorm
from scipy.optimize import curve_fit
from scipy import integrate
import os

def lognorm_dist(x, mu, sigma, scale1):
    return scale1 * lognorm.pdf(x, sigma, scale=np.exp(mu))

def n_modal_lognorm(x, *params):
    output = 0
    n = len(params) // 3
    for i in range(n):
        output += lognorm_dist(x, *params[i*3:(i+1)*3])
    return output

def cut_small_edge(x, y, th=0.1):
    cut_index = [0,len(x)]
    for i in range(len(x)):
        if x[i] > th:
            cut_index[0] = i
            break
    for i in range(len(x)-1,0,-1):
        if x[i] > th:
            cut_index[1] = i+1
            break
    return x[cut_index[0]:cut_index[1]], y[cut_index[0]:cut_index[1]]

def square_error_d2f(x, y, func, param):
    y2 = func(x, *param)
    sse = np.sum((y-y2)**2)
    sst = np.sum((y-np.mean(y))**2)
    return 1 - sse/sst

def fit_nlognorm(sample, n,  output_path, param_path,
                initial_guess = [1,1,0.01]): # もし、フィッティングが上手くいかなければ調整
    # Read data from the text file
    sample_name = os.path.basename(sample)
    x_values = []
    y_values = []
    with open(sample, 'r') as file: # 読み込むファイルを指定
        lines = file.readlines()
        for line in lines:
            split_line = line.split(',')
            x_values.append( float(split_line[0]) )
            y_values.append( float(split_line[1]) )

    y_values, x_values  = cut_small_edge(np.array(y_values), np.array(x_values), th=0.01)
    # y_values /= integrate.trapz(y_values, x_values)

    initial_p0 = []
    for i in range(n):
        this_p0 = initial_guess.copy()
        this_p0[0] = this_p0[0] * (i+1)
        initial_p0.append(this_p0)

    # Fit the function to the data
    params, _ = curve_fit(n_modal_lognorm, x_values, y_values, p0=initial_p0)

    # Create the x values for the fitted function
    x_fit = np.logspace(np.log10(min(x_values)), np.log10(max(x_values)), 100)
    # Compute the y values for the fitted function
    y_fit = n_modal_lognorm(x_fit, *params)


    # Create the graph
    plt.plot(x_values, y_values, 'b:', label="Original") # Plot the data
    # Plot the fitted function
    plt.plot(x_fit, y_fit, 'r-', label='Fitted')

    for i in range(n):
        y_fit_s = lognorm_dist(x_fit, *params[(3*i):(3*i+3)])
        plt.plot(x_fit, y_fit_s, linestyle='--', label=f'{i+1}-logNorm')

    # Set x-axis to logarithmic scale
    plt.xscale('log')

    # Add labels and title
    plt.xlabel(r'Diameter / $\rm\mu$m')
    plt.ylabel('Probability Density (%)')
    plt.title(f'SPS distribution of {sample_name.replace(".txt", "")}')

    # Add legend
    plt.legend()

    # Save the graph
    plt.savefig(output_path, dpi=300)
    plt.savefig(output_path.replace('.png', '.pdf'))

    # Display the graph
    plt.show()

    # Define the string to be saved
    output_string = f"weight, {n}\n"
    error = square_error_d2f(x_values, y_values, n_modal_lognorm, params)
    error = round(error, 3)
    output_string += f"error(R^2), {error}\n"
    output_string += "n, log(mu), mu, sigma, scale\n"
    for i in range(n):
        params_i = params[(3*i):(3*i+3)]
        output_string += (f"{i+1}, {round(params_i[0], 3)}, {round(np.exp(params_i[0]), 3)}, {round(params_i[1], 3)}, {round(params_i[2], 3)}\n")

    print(f"error(R^2): {error}")
    # Open the file in write mode ('w')

    with open(param_path, 'w') as file: # 保存先のファイル名を指定
        # Write the string to the file
        file.write(output_string)

# ====ここまで変更しない====

#@markdown  **ディレクトリー名**
output_dir = 'FN3T' #@param {"type":"string"}

# ディレクトリー作成
os.makedirs("data", exist_ok=True)
os.makedirs(os.path.join("data", output_dir), exist_ok=True)
os.makedirs("graph", exist_ok=True)
os.makedirs(os.path.join("graph", output_dir), exist_ok=True)
os.makedirs("Fitting_Params", exist_ok=True)
os.makedirs(os.path.join("Fitting_Params", output_dir), exist_ok=True)

print("===ディレクトリーを作成しました===")

In [ ]:
#@title ファイル確認
#@markdown 1. **data/[ディレクトリー名]/** にデータ(txtファイル)をアップロードしてください
#@markdown 2. 実行ボタンを押して、ファイル名一覧を確認してください

file_names = os.listdir( os.path.join('data/', output_dir) )
print("===ファイル名一覧===")
for i, file_name in enumerate(file_names):
    print(f"{i}: {file_name}")

In [ ]:
#@title 解析するファイル設定
# @markdown  **すべてのファイルでフィッティング**
is_all = False # @param {"type":"boolean"}
# @markdown  **番号でフィッティング**
is_series = True # @param {"type":"boolean"}
# @markdown **ファイルの番号を入力** (例: 1,2,5)
file_number = "2" #@param {"type":"string"}
number_list = [int(i) for i in file_number.split(',')]

error_double = False
if is_all and is_series:
  error_double = True
  raise ValueError("is_allとis_seriesは同時にTrueにできません")

if not is_all and not is_series:
  error_double = True
  raise ValueError("is_allとis_seriesは同時にFalseにできません")

if error_double:
  raise ValueError("エラーが発生しました")
else:
  if is_all:
    print("===すべてのファイルを解析します===")
  elif is_series:
    print("===入力ファイル名一覧===")
    for i, num in enumerate(number_list):
      print(f"{i}: {file_names[num]}")



In [ ]:
#@title パラメーター設定
#@markdown **フィッティングの重み(分布の個数)**
weight = 1 # @param {"type":"slider","min":1,"max":5,"step":1}
file_names = os.listdir( os.path.join('data/', output_dir) )

#@markdown **初期値(必ず3つの数値配列で！)** (例[0.1, 0.1, 0.5])
Initial_guess = [0.1, 0.1, 0.5] # @param
this_file_names = []

if is_all:
  this_file_names = file_names

if is_series:
  for num in number_list:
    this_file_names.append(file_names[num])

print(f"weight = {weight}\n=================")

for i, file_name in enumerate(this_file_names):
  print(i, file_name)
  Output_path = os.path.join('graph/', output_dir, file_name.replace('.txt', '_fit.png'))
  Param_path = os.path.join('Fitting_Params/', output_dir, file_name.replace('.txt', '_params.txt'))
  fit_nlognorm(os.path.join('data/', output_dir, file_name), n=weight, output_path=Output_path, param_path=Param_path, initial_guess = Initial_guess)
  print("=================")



In [ ]:
#@title 結果を削除

#@markdown **出力結果を削除しますか？**
is_clear_outputs = False #@param {"type":"boolean"}
#@markdown **データも削除しますか？**
is_clear_data = False #@param {"type":"boolean"}

if is_clear_outputs:
  os.system("rm -rf graph/*")
  os.system("rm -rf Fitting_Params/*")
  os.makedirs(os.path.join("graph", output_dir), exist_ok=True)
  os.makedirs(os.path.join("Fitting_Params", output_dir), exist_ok=True)
  print("===出力結果を削除しました===")
  if is_clear_data:
    os.system("rm -rf data/*")
    print("===データも削除しました===")
else:
  print("===削除しません===")